**Getting data from teslapy, and processing the data**

In [ ]:
import teslapy
import time
i=0

from datetime import datetime

now = datetime.now()

timestamp = datetime.timestamp(now)




while i<172800:
    try:

        text_file = open(f"Wonmik Driving{timestamp}.txt", "a")

        with teslapy.Tesla('YOUR TESLA EMAIL ADRESS') as tesla:
            starttime = time.time()

            vehicles = tesla.vehicle_list()
            now = datetime.now()
            print(now)
            text_file.write(str(vehicles[0].get_vehicle_data()))
            text_file.write('\n')
            text_file.write(',')
            text_file.write('\n')
            time.sleep(3)
            i=i+1
    except:
        print('passed')
        pass
text_file.close()

In [ ]:
def make_it_json(FileNa):

    f= open(FileNa+'.txt')
    json=open(FileNa+'.json', "w")
    json.write['[\n']
    index=0
    for i in f:
        json.write(i)
        index=index+1
    json.write['\n]']

    f.close()
    json.close()
make_it_json('Wonmik Driving_name')

#do it with glob.glob and for loop, it is easier


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)
df_1=pd.read_json('Wonmik Driving 10_43_24 ___ 10_49_05txt.json')[['display_name','state','charge_state','climate_state','drive_state','gui_settings','vehicle_config','vehicle_state']]
df_2=pd.read_json('Wonmik Driving  11_05_51___ 11_37_10.json')[['display_name','state','charge_state','climate_state','drive_state','gui_settings','vehicle_config','vehicle_state']]
df_3=pd.read_json('Wonmik Driving  12_12_25___ 12_16_10.json')[['display_name','state','charge_state','climate_state','drive_state','gui_settings','vehicle_config','vehicle_state']]
df_4=pd.read_json('Wonmik Driving 12_17_32 ___  12_21_34.txt')[['display_name','state','charge_state','climate_state','drive_state','gui_settings','vehicle_config','vehicle_state']]
df_5=pd.read_json('Wonmik Driving12_21_45___end - Copy.json')[['display_name','state','charge_state','climate_state','drive_state','gui_settings','vehicle_config','vehicle_state']]

df=pd.concat([df_1,df_2,df_3,df_4,df_5]).reset_index(drop=True)
for i in df.index:
    df.loc[i,'timestamp']=df.loc[i,'vehicle_state']['timestamp']
df=df.sort_values(by=['timestamp']).reset_index(drop=True)

In [ ]:
for i in df.index:
#     df.loc[i,'timestamp']=df.loc[i,'vehicle_state']['timestamp']
    df.loc[i,'temp_setting']=df.loc[i,'climate_state']['driver_temp_setting']
    df.loc[i,'in_temp']=df.loc[i,'climate_state']['inside_temp']
    df.loc[i,'out_temp']=df.loc[i,'climate_state']['outside_temp']
    df.loc[i,'battery']=df.loc[i,'charge_state']['battery_level']
    df.loc[i,'charge_rate']=df.loc[i,'charge_state']['charge_rate']
    df.loc[i,'charging_state']=df.loc[i,'charge_state']['charging_state']
    try:
        df.loc[i,'latitude']=df.loc[i,'drive_state']['latitude']
        df.loc[i,'longitude']=df.loc[i,'drive_state']['longitude']
    except:
        df.loc[i,'latitude']=''
        df.loc[i,'longitude']=''

    try:
        df.loc[i,'heading']=df.loc[i,'drive_state']['heading']
    except:
        df.loc[i, 'heading']=''
    try:
        df.loc[i,'shift']=df.loc[i,'drive_state']['shift_state']
    except:
        df.loc[i, 'shift']=''
    try:
        df.loc[i,'speed']=df.loc[i,'drive_state']['speed']
    except:
        df.loc[i, 'speed']=''
        
    try:
        df.loc[i,'power']=df.loc[i,'drive_state']['power']
    except:
        df.loc[i, 'power']=''    
    df.loc[i,'gui_charge_rate_units']=df.loc[i,'gui_settings']['gui_charge_rate_units']
    df.loc[i,'gui_distance_units']=df.loc[i,'gui_settings']['gui_distance_units']
    df.loc[i,'gui_temperature_units']=df.loc[i,'gui_settings']['gui_temperature_units']
    try:
        df.loc[i,'odometer']=df.loc[i,'vehicle_state']['odometer']
    except:
        df.loc[i, 'odometer']=''    


In [ ]:
df=df[df.latitude!='']

df=df[df.latitude.notna()]
# df=df[df['charging_state']=='Disconnected']

for i in df.index:
    df.loc[i, 'timestamp']=int(str(df.loc[i, 'timestamp'])[:10])
# int(str(df.timestamp[0])[:10])
df=df.sort_values(by=['timestamp']).reset_index(drop=True)

dict_index={}
for n in df.index:
    dict_index[int(df.loc[n,'timestamp'])]=n
    
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="won")
for i in df.index:
    raw_data=geolocator.reverse(str((df.loc[i,'latitude']))+","+str((df.loc[i,'longitude']))).raw
    try:
        
        df.loc[i,'city']='City: '+raw_data['address']['city']
    except:
        df.loc[i,'city']=''
    if raw_data['address']['country']=='Canada':
        df.loc[i,'state']='Province: '+raw_data['address']['state']
    else:
        df.loc[i,'state']='State: '+raw_data['address']['state']
    try:
        
        df.loc[i,'amenity']=raw_data['address']['amenity']
    except:
        df.loc[i,'amenity']=''
    try:
        
        df.loc[i,'road']='Road: '+raw_data['address']['road']
    except:
        df.loc[i,'road']=''
        
print(2)
 

        
df=df. T. drop_duplicates().T


In [ ]:
import datetime
import pytz
from tzwhere import tzwhere
from dateutil import tz

for idxx in df.index:
    if df.loc[idxx, 'state']=='Province: Alberta':
        timezone = 'Canada/Mountain'
    else:
        timezone= 'Canada/Pacific'
        
    tz_name = pytz.timezone(timezone)
    
    
    timestamp= df.loc[idxx , 'timestamp']
    
    df.loc[idxx, 'local time']=datetime.datetime.fromtimestamp(timestamp , tz = tz.gettz(str(tz_name)))

    
#########################since tzwhere takes too much time, I set up the timezone manually. faster this way



# def local_time(row):
#     timezone=tzwhere.tzwhere().tzNameAt((row.latitude), (row.longitude))
#     tz_name = pytz.timezone(timezone)

#     timestamp=row.timestamp
    
#     return datetime.datetime.fromtimestamp(timestamp , tz = tz.gettz(str(tz_name)) )

  
# df['local time']=df.apply(local_time, axis=1)


In [ ]:
df=df[['display_name','timestamp','temp_setting','in_temp','out_temp','battery','charge_rate','charging_state','latitude','longitude','heading','shift','speed','power','gui_charge_rate_units','gui_distance_units','gui_temperature_units','odometer','city','amenity','state','road','local time']]
df=df. T. drop_duplicates().T
df.to_csv('wonmik_driving_complete_f.csv')

In [ ]:
import pandas as pd
df=pd.read_csv('/content/wonmik_driving_complete_f.csv').set_index('Unnamed: 0')
df=df[1:]
# df.regen=df.regen.fillna(0)
df.power=df.power.fillna(0)
df.state=df.state.fillna('')
df.city=df.city.fillna('')
df.road=df.road.fillna('')
df.amenity=df.amenity.fillna('')
# df=df[df['shift']=='D']
df.speed=df.speed.fillna(0)
df.columns
df
df=df.loc[list(df['local time'][:598].index)+list(df['local time'][602:].index)].sort_values(by='odometer').reset_index()


In [ ]:
for i in df.index:

    if  abs(df.loc[i,'latitude']-df.loc[i+1,'latitude'])>0.07:
        print (i)
#to find out if there is any error samething with longitude

**use Matplotlib to show the vehicle info**

In [ ]:

def main():
    from matplotlib.transforms import Affine2D
    import mpl_toolkits.axisartist.floating_axes as floating_axes
    import matplotlib.pyplot as plt_power
    import matplotlib.pyplot as plt
    from matplotlib import pyplot, transforms
    import gc



    import cv2
    import glob
    import os

    list_file=[]

    for i in glob.glob("/content/drive/MyDrive/figured2/*.png"):
        
        list_file.append(int(i.split('figured2/')[1].split('_')[0][1:]))
        
    try:

        print(max(list_file))
    except:
        pass




    collected_indx=0

    mon_d={1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
    df_distance = df.loc[df[(df.amenity!='')][['amenity']].drop_duplicates().index][['amenity', 'odometer']]
    odometer_ini=df.odometer.min()
    odometer_fin=df.odometer.max()
    df_Charges=pd.read_csv('Charges.csv')
    df_Charges=df_Charges.loc[df_Charges.Location.dropna().index]
    plt.rcParams["figure.figsize"] = [40,4]
    fig, axes = plt.subplots(ncols=3,nrows=3,gridspec_kw={'width_ratios': [138, 14,138],'height_ratios':[1,40,1]})
    pls=[]
    for i in df.index:
        if df.loc[i,'gui_distance_units']=='km/hr':
            Kph=df.loc[i,'speed']
            Mph=Kph/1.609
        elif df.loc[i,'gui_distance_units'] =='mi/hr':
            Mph=df.loc[i,'speed']
            Kph=Mph*1.609
        pls.append(Kph)
    max_speed = max(pls)
    max_regen=max(df.power*-1)
    max_power=max(df.power)
    units="DEGREES" 
    l_shft=['P','N','R','D']
    def C2F(temp):
        return temp*9/5+32
    def F2C (temp):
        return (temp-32)*5/9
    def clamp(x):
        return max(0, min(int(x), 255))
    def rgb_to_hex(r,g,b): 
        def clamp(x):
            return max(0, min(int(x), 255))
        r= max(0, min(r, 255))    
        g=max(0, min(g, 255))
        b=max(0, min(b, 255))
        return"#{0:02x}{1:02x}{2:02x}".format(clamp(r), clamp(g), clamp(b))

    def set_color(T):
        r=255
        g=255
        b=255
        diff_min=-50
        diff_max= 10
        if df.loc[i,'gui_temperature_units']=='C':
            pass
        else:
            diff_min=-58
            diff_max=50

        if T<20:
            r=r-(T-20)/diff_min*255
            g=g-(T-20)/diff_min*72
            r=int(r)
            g=int(g)
            b=int(b)
        elif T>20:
            # g=g-20
            # b=b-20        

            g=g-(T-20)/diff_max*218
            b=b-(T-20)/diff_max*235
            r=int(r)
            g=int(g)
            b=int(b)

        else:
            g=g
            # g=g-20
            # b=b-20
        return rgb_to_hex(r, g, b)

    def set_color_regen():
        r=105
        g=255 
        b=167
        r_to=0
        g_to=200
        b_to=255
        change_rate=df.loc[i,'power']*-1/max_regen
        r=int(r+(r_to-r)*change_rate)
        g=int(g+(g_to-g)*change_rate)
        b=int(b+(b_to-b)*change_rate)
        return"#{0:02x}{1:02x}{2:02x}".format(clamp(r), clamp(g), clamp(b))

    def set_color_power():
        r=30 
        g=0 
        b=0
        r_to=255
        g_to=0
        b_to=0
        change_rate=df.loc[i,'power']/max_power
        r=int(r+(r_to-r)*change_rate)
        g=int(g+(g_to-g)*change_rate)
        b=int(b+(b_to-b)*change_rate)
        return"#{0:02x}{1:02x}{2:02x}".format(clamp(r), clamp(g), clamp(b))




    def odometer_to_x(x,x_shift):
        return (x_shift+0.172279889+0.889229273 -(0.172279889+0.889229273)*(x-odometer_ini)/(odometer_fin-odometer_ini))




    for i in df.index:
    # for i in range(start ,start+10700):


        if i in list_file:
            pass
        else:
            try:

                
                  

                power=[int(df.loc[i,'power'])]
                # regen=[int(df.loc[i,'regen'])]
                regen=[num*-1 for num in power]
                battery=[int(df.loc[i,'battery'])]

                x_shift=df.loc[i,'longitude']
                y_shift=df.loc[i,'latitude']

                x_list=[]
                y_list=[]
                x_list_togo=[]
                y_list_togo=[]
                pl_x=[]
                pl_y=[]
                pl_n=[]




            ################################################################
                fig, axes = plt.subplots(ncols=3,nrows=3,gridspec_kw={'width_ratios': [138, 14,138],'height_ratios':[1,40,1]})
                for pr in [0,2]:
                    axes[0,pr].barh(['power'], [100], align='center', color='#f0e6e6')
                    axes[0,pr].barh(['power'], power, align='center', color=set_color_power())
                    axes[2,pr].barh(['regen'], [100], align='center', color='#e6f0e8')
                    axes[2,pr].barh(['regen'], regen, align='center', color=set_color_regen())
                axes[1,1].bar(['Battery'], [100], align='center', color='#e6f0e8')
                axes[1,1].bar(['Battery'], battery, align='center', color='#69ffa7')

                axes[0,0].set_xlim([0,100])
                axes[0,2].set_xlim([0,100])
                axes[2,0].set_xlim([0,50])
                axes[2,2].set_xlim([0,50])
                axes[1,1].set_ylim([0,100])

                axes[0,0].invert_xaxis()
                axes[2,0].invert_xaxis()
                

                for h in range(3):
                    for w in range(3):
                        axes[h,w].spines['top'].set_visible(False)
                        axes[h,w].spines['left'].set_visible(False)
                        axes[h,w].spines['right'].set_visible(False)
                        axes[h,w].spines['bottom'].set_visible(False)
                        axes[h,w].get_xaxis().set_visible(False)
                        axes[h,w].get_yaxis().set_visible(False)

                plt.figtext(0.5125, 0.5,str(battery[0])+'%' , ha='center', va='center',c='w',fontsize=20*2)
                if power[0]>0:
                    plt.figtext(0.5125, 0.87,'-Power-' , ha='center', va='center',c=set_color_power(),fontsize=20*2)
                else:
                    plt.figtext(0.5125, 0.87,'-Power-' , ha='center', va='center',c='#f0e6e6',fontsize=20*2)
                if df.loc[i, 'charging_state']=='Disconnected':
                    charging_type='+Regen+'
                else:
                    charging_type='+Charging+'

                if regen[0]>0:
                    plt.figtext(0.5125, 0.116,charging_type , ha='center', va='center', c=set_color_regen(),fontsize=20*2)
                else:
                    plt.figtext(0.5125, 0.116,charging_type , ha='center', va='center', c='#e6f0e8',fontsize=20*2)




                if df.loc[i,'gui_distance_units']=='km/hr':
                    Kph=df.loc[i,'speed']
                    Mph=Kph/1.609
                elif df.loc[i,'gui_distance_units'] =='mi/hr':
                    Mph=df.loc[i,'speed']
                    Kph=Mph*1.609
                Mph=int(Mph)
                Kph=int(Kph)    
                if df.loc[i, 'state'].split(':')[0]=='Province':

                    infospeed=str(int(Mph))+' MI/H'
                    mainspeed=str(Kph)+' KM/H'
                else:
                    mainspeed=str(int(Mph))+' MI/H'
                    infospeed=str(Kph)+' KM/H'




                speed_now=Kph+Kph//7
                rd_i=400
                gn_i=400
                bl_i=400
                rd=84
                gn=161
                bl=255
                l_s=[Kph%7]
                l_r=[]
                l_g=[]
                l_b=[]

                for speedindex in range(int(Kph//1)):

                    l_s=[Kph%1+speedindex]+l_s
                    l_r=l_r+[rd+(rd_i-rd)/(max_speed//1)*speedindex]
                    l_g=l_g+[gn+(gn_i-gn)/(max_speed//1)*speedindex]
                    l_b=l_b+[bl+(bl_i-bl)/(max_speed//1)*speedindex]

                for speedindex in range(int(Kph//1)):
                    try:
                        axes[1,2].barh(['1','2','3','4','5','Speed','6','7','8'], [0]*5+[l_s[speedindex]]+[0]*3 , align='center', color=rgb_to_hex(int(l_r[speedindex]),  int(l_g[speedindex]),int(l_b[speedindex]) ) )
                    except:
                        axes[1,2].barh(['1','2','3','4','5','Speed','6','7','8'], [0]*5+[l_s[speedindex]]+[0]*3 , align='center', color=rgb_to_hex(255,255,255)  )


            # ################################################################  distance
            #     axes[1,0].barh(['1','2','3','4','5','Distance','6','7','8'], [0]*5+[l_s[speedindex]]+[0]*3 , align='center', color=rgb_to_hex(255,255,255)  )




                if l_r==[]:
                    speed_color='w'
                    pass
                else:
                    speed_color=rgb_to_hex(int(l_r[-1]),  int(l_g[-1]),int(l_b[-1]))

                    plt.figtext(0.80, 0.3,mainspeed , ha='center', va='center',c=speed_color,fontsize=55*2,style='italic')
                    plt.figtext(0.84, 0.7,infospeed , ha='center', va='center',c=speed_color,fontsize=25*2,style='italic')

                for shft in range(4):
                    if l_shft[shft]==df.loc[i,'shift']:
                        plt.figtext(0.348+0.03*shft, 0.3,l_shft[shft], ha='center', va='center',c="#34c3eb",fontsize=45*2,style='italic')
                    else:
                        plt.figtext(0.348+0.03*shft, 0.3,l_shft[shft], ha='center', va='center',c='#e0e0e0',fontsize=45*2,style='italic')



                axes[1,2].set_xlim([0,max_speed])




            #################################### temperature display
                if df.loc[i,'gui_temperature_units']=='C':
                    plt.figtext(0.5125-0.1, 1-0.07,('Out: '+str(int(df.loc[i,'out_temp']))+'°C') , ha='center', va='center', c=set_color(df.loc[i,'out_temp']),fontsize=30)
                    plt.figtext(0.5125+0.1, 1-0.07,('Out: '+str(int(df.loc[i,'out_temp']*9/5+32))+'°F') , ha='center', va='center', c=set_color(df.loc[i,'out_temp']),fontsize=30)
                    plt.figtext(0.5125-0.1, 1-0.2,(' In: '+str(int(df.loc[i,'in_temp']))+'°C') , ha='center', va='center', c=set_color(df.loc[i,'in_temp']),fontsize=30)
                    plt.figtext(0.5125+0.1, 1-0.2,(' In: '+str(int(df.loc[i,'in_temp']*9/5+32))+'°F') , ha='center', va='center', c=set_color(df.loc[i,'in_temp']),fontsize=30)
                else:

                    plt.figtext(0.5125+0.1, 1-0.07,('Out: '+str(int(F2C(df.loc[i,'out_temp'])))+'°C') , ha='center', va='center', c=set_color(df.loc[i,'out_temp']),fontsize=30)
                    plt.figtext(0.5125-0.1, 1-0.2,(' In: '+str(int(F2C(df.loc[i,'in_temp'])))+'°C') , ha='center', va='center', c=set_color(df.loc[i,'in_temp']),fontsize=30)
                    plt.figtext(0.5125+0.1, 1-0.07,('Out: '+str(int((df.loc[i,'out_temp'])))+'°F') , ha='center', va='center', c=set_color(df.loc[i,'out_temp']),fontsize=30)
                    plt.figtext(0.5125+0.1, 1-0.2,(' In: '+str(int(df.loc[i,'in_temp']))+'°F') , ha='center', va='center', c=set_color(df.loc[i,'in_temp']),fontsize=30)


            ################################ date time display

                DnT=str((df.loc[i,'local time'])).split('-')
                DnT[1]
                time=DnT[-2].split(' ')[1][:-3]
                date=mon_d[int(DnT[1])] +'. '+DnT[-2].split(' ')[0]+'. '+str((df.loc[i,'local time'])).split('-')[0]+'.'        
                plt.figtext(0.611, 0.35,time , ha='center', va='center',c='w',fontsize=35*2,style='italic')
                plt.figtext(0.6123, 0.20,date , ha='center', va='center',c='w',fontsize=13.43*2,style='italic')



                df_h2=df[(-0.18<df.longitude-x_shift)&(df.longitude-x_shift<1)&(-0.21<df.latitude-y_shift)&(df.latitude-y_shift<0.14)]
                df_h2_Charger=df_Charges[(-0.18<df_Charges.Longitude-x_shift)&(df_Charges.Longitude-x_shift<1)&(-0.21<df_Charges.Latitude-y_shift)&(df_Charges.Latitude-y_shift<0.14)]


                axes[1,0].set_xlim([x_shift-0.172279889,x_shift+0.172279889+0.889229273])
                axes[1,0].set_ylim([y_shift-0.208279889,y_shift+0.136279889])


                for k in df_h2.index:
                    x=df_h2.loc[k,'longitude']
                    y=df_h2.loc[k,'latitude']

                    if k>=i:
                        x_list_togo.append(x)
                        y_list_togo.append(y)


                    x_list.append(x)
                    y_list.append(y)



                axes[1,0].plot(df_h2_Charger['Longitude'], df_h2_Charger['Latitude'],c='#34c3eb',linewidth=2.5*2)

                axes[1,0].plot(x_list, y_list,c='#34c3eb',linewidth=2.5*2)
                axes[1,0].plot(x_list_togo, y_list_togo, c='w' ,linewidth=1.8*2)
                maxnum=1000
                term=10
                for num in range(0,maxnum,term):
                    if i-num>=term+2:
                        axes[1,0].plot(list([odometer_to_x( df.loc[i-num,'odometer'], x_shift) ,odometer_to_x(df.loc[i-term-num,'odometer'], x_shift )]) , [y_shift]*2, c="#{0:02x}{1:02x}{2:02x}".format(clamp(int(255-203/maxnum*num)), clamp(int(255-60/maxnum*num)), clamp(int(255-20/maxnum*num))) ,linewidth=2.5*2, alpha=1-num/maxnum )

                axes[1,0].scatter(     [odometer_to_x( charg_loc, x_shift)  for charg_loc in df_Charges['Odometer (mi)'] ], [y_shift]*10 , color='#e6f0e8' ,s=10  )
            #     axes[1,0].plot(df_h2_Charger['Latitude'], df_h2_Charger['Longitude'],'o',c= color="#{0:02x}{1:02x}{2:02x}".format(clamp(int(230-125/max_dis*(max_dis-char_dis))), clamp(int(240-15/max_dis*(max_dis-char_dis))), clamp(int(232-65/max_dis*(max_dis-char_dis)))))
                Charger_found=False

                for charg in df_Charges.index:
                    charge_where=df_Charges.loc[charg,'Odometer (mi)']
                    char_dis=abs(df.odometer[i]-charge_where)
                    max_dis=3
                    if char_dis<max_dis:
                        plt.figtext(0.25, 0.21,df_Charges.loc[charg,'Location'] , ha='center', va='center',c='#69ffa7',fontsize=20*2,style='italic')
                        axes[1,0].scatter(     [odometer_to_x( charge_where, x_shift)], [y_shift] ,c='#69ffa7' ,s=100)

                        Charger_found=True
                try:
                    place=df.loc[i, 'amenity']
                    if place!='':


                        if Charger_found==False:
                            plt.figtext(0.25, 0.21,'Place: '+place , ha='center', va='center',c='#34c3eb',fontsize=20*2,style='italic')


                except:
                    pass

                try:

                    char_dis=abs(min(df_h2_Charger['Odometer (mi)']-df.loc[i, 'odometer']))
                    max_dis=500    
                    axes[1,0].plot(df_h2_Charger['Longitude'], df_h2_Charger['Latitude'],'o',c= "#{0:02x}{1:02x}{2:02x}".format(clamp(int(230-125/max_dis*(max_dis-char_dis))), clamp(int(240-15/max_dis*(max_dis-char_dis))), clamp(int(232-65/max_dis*(max_dis-char_dis)))), alpha=(max_dis-char_dis)/max_dis)
                except:
                    
                    pass




                axes[1,0].text(x_shift+0.01,y_shift+0.01 ,f"{df.loc[i,'road']}\n{df.loc[i,'city']}\n{df.loc[i,'state']}\n({int(df.loc[i,'latitude']*10000)//10/1000}, {int(df.loc[i,'longitude']*10000)//10/1000})" ,  c='w',  style='italic', alpha=0.8,  fontsize=18)
                axes[1,0].text(odometer_to_x( df.loc[i,'odometer'], x_shift),y_shift-0.03 ,str(int(df.loc[i,'odometer']-odometer_ini))+'MI  '+str(int(1.609*(df.loc[i,'odometer']-odometer_ini)))+'KM'  ,  c='w',  style='italic', alpha=0.8,  fontsize=10)






                # plt.axis('off')
                # plt.show()


                time=df.loc[i,'timestamp']
                plt.savefig(f'/content/drive/MyDrive/figured2/{"%06d" % (i)}_{time}.png', transparent=True)
                plt.close()
            except:
                plt.close()
                pass




            del mainspeed
            del infospeed
            del speed_now
            del rd_i
            del gn_i
            del bl_i
            del rd
            del gn
            del bl
            del l_s
            del l_r
            del l_g
            del l_b
            # del place
        #     del df_h2
        #     del plt
            del Mph
            del Kph
            del x_shift
            del y_shift
            del x_list
            del y_list
            del x_list_togo
            del y_list_togo
            del pl_x
            del pl_y
            del pl_n
        #     del df_h2
            # if i//100>collected_indx:
            #     print(i,'cleaning...')
            #     gc.collect()
            #     print(i,'cleaned')

            #     collected_indx=collected_indx+1



In [ ]:
import gc
gc.collect()
main()

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/gdrive


**Combining Matplotlib display and the original frames from the vid, and saving it frame by frame**

**I went for this approach because I'm using multiple accounts and multiple instences. normally, it would be faster to do all those steps at once**

In [ ]:
from PIL import Image
import os
import glob
dict_fig={ int(i.split('_')[-1].split('.')[0]):i for i in glob.glob("/content/drive/MyDrive/figured2/*.png")}

for k in glob.glob("/content/drive/MyDrive/teslapy_back to_OR/*.mp4"):
        bases=glob.glob(k[:-4]+'/*.jpg')
        dir='/content/drive/MyDrive/'+k.split('/')[-1].split('.')[0]
        try:
            os.mkdir(dir)
        except:
            pass
            
        print(dir)
        for i in range(len(bases)):
            timestamp_cur=int(k.split('/')[-1].split('_')[0])+float(bases[i].split('/')[-1][:-4])*30/1000
            if os.path.isfile(dir+'/'+str(int(round(timestamp_cur)))+'.jpg'):
                print( dir+'/'+str(int(round(timestamp_cur)))+'.jpg',', passed!')
                pass
            else:

                  
                timestamp_to_use= min(dict_fig.keys(), key=lambda x:abs(x-timestamp_cur))
                background = Image.open(bases[i])
                if abs(timestamp_cur-timestamp_to_use)<6:
                        foreground=Image.open(dict_fig[timestamp_to_use])


                        foreground = foreground.resize((4780, 480), Image.ANTIALIAS)
                        background.paste(foreground, (-540, 1650), foreground)

                        background.save(dir+'/'+str(int(round(timestamp_cur)))+'.jpg')
                else:
                        background.save(dir+'/'+str(int(round(timestamp_cur)))+'.jpg')

        import gc 
        gc.collect()

**making it a vid again**

In [ ]:
pip install moviepy

In [ ]:
!pip3 install imageio==2.4.1



In [ ]:
!pip install --upgrade imageio-ffmpeg



In [ ]:
import os
import glob
import moviepy.video.io.ImageSequenceClip
from google.colab import files
image_folder='folder_with_images'
fps=30
name='1668879674_1668881163'

image_files = glob.glob('/content/drive/MyDrive/'+name+'/*.jpg')
image_files.sort()
clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)
clip.write_videofile(f'{name}.mp4')
files.download(f'{name}.mp4')
